<a href="https://colab.research.google.com/github/UrmilaJasiwar/Face_Mask_Detection/blob/master/Face_image_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

#predicting the image
def mask_image():
  mask_cont = no_mask_cont = 0
  prototxtPath ='/content/drive/My Drive/face_detector/deploy.prototxt'
  weightsPath = '/content/drive/My Drive/face_detector/res10_300x300_ssd_iter_140000.caffemodel'
  net = cv2.dnn.readNet(prototxtPath, weightsPath)
  model = load_model('/content/drive/My Drive/Face_model/model-ep020-loss0.016-val_loss0.015.h5')
  image = cv2.imread('/content/drive/My Drive/face_detector/e35d1b65f1ee4f7f9a3d75a2662cc32d_18.jpg')
  orig = image.copy()
  (h, w) = image.shape[:2]
  blob = cv2.dnn.blobFromImage(image, 1.0, (299, 299),
		(104.0, 177.0, 123.0))
  net.setInput(blob)
  detections = net.forward()
  for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      (startX, startY, endX, endY) = box.astype("int")
      (startX, startY) = (max(0, startX), max(0, startY))
      (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
      face = image[startY:endY, startX:endX]
      face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
      face = cv2.resize(face, (299, 299))
      face = img_to_array(face)
      face = preprocess_input(face)
      face = np.expand_dims(face, axis=0)
      (mask, withoutMask) = model.predict(face)[0]
      if mask > withoutMask:s
        label = "Mask"
        mask_cont += 1
      else:
        label = "No Mask"
        no_mask_cont += 1
      color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
      cv2.putText(image, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
      cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
  cv2.putText(image, "People with Mask : "+str(mask_cont) , (startX-20, startY - 150),
				cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
  cv2.putText(image, "People without Mask : "+str(no_mask_cont) , (startX-20, startY - 120),
				cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
  cv2.imwrite("Output2.jpg", image)
 
mask_image()